Use another dataset and merge it with the dataset used for calculating the Wilson Score so that we can get the tags needed for content based collaborative filtering

In [2]:
import pandas as pd 
import numpy as np
import pandas as pd
import requests
import time
import json
import os
from tqdm import tqdm
import difflib

In [2]:
# load the dataframe used for wilson scoring
df_wilson = pd.read_json('game_stats_wilson.json', orient='records', lines=True)

In [3]:
df_wilson.head()

,appid,game,total_votes,upvotes,downvotes,wilson_score
0,550,Left 4 Dead 2,1309,1309,0,0.997074
1,644560,Mirror,925,925,0,0.995864
2,264710,Subnautica,1333,1332,1,0.995763
3,294100,RimWorld,1294,1293,1,0.995635
4,413150,Stardew Valley,2240,2236,4,0.995417


In [4]:
df_wilson.shape

(27315, 6)

In [5]:
# just need the appid and game
df_wilson_games = df_wilson[['appid', 'game']]

In [6]:
df_wilson_games.head()

,appid,game
0,550,Left 4 Dead 2
1,644560,Mirror
2,264710,Subnautica
3,294100,RimWorld
4,413150,Stardew Valley


In [7]:
# load csv into df
df_tags = pd.read_csv('dataset/steam_games-40k.csv')

In [8]:
df_tags

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,app,Rocksmith® 2014 Edition – Remastered – Sabaton...,NaN,NaN,NaN,"Feb 12, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",NaN,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7,

In [9]:
# Convert the 'genre' column from a comma-separated string to a list
df_tags['genre'] = df_tags['genre'].str.split(',')

# Display the updated DataFrame
df_tags.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,[Action],"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"[Action, Adventure, Massively Multiplayer]",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"[Action, Adventure, Strategy]",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"[Action, Adventure, Massively Multiplayer]",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"[Action, Free to Play, Massively Multiplayer, ...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


I grab the name and genre columns

In [ ]:
# take the genres
df_tags_important = df_tags[['name', 'genre']]

In [11]:
df_tags_important.head()

,name,genre
0,DOOM,[Action]
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"[Action, Adventure, Massively Multiplayer]"
2,BATTLETECH,"[Action, Adventure, Strategy]"
3,DayZ,"[Action, Adventure, Massively Multiplayer]"
4,EVE Online,"[Action, Free to Play, Massively Multiplayer, ..."


Merge both datasets but only keep the games that belong to df_wilson 

In [13]:
# Ensure game names are case-insensitive and merge datasets
df_wilson['game_lower'] = df_wilson['game'].str.lower()
df_tags_important['name_lower'] = df_tags_important['name'].str.lower()

# Merge datasets on the lowercased game name
merged_df = pd.merge(
    df_wilson,
    df_tags_important,
    left_on='game_lower',
    right_on='name_lower',
    how='left'
)

# Drop unnecessary columns after merging
merged_df = merged_df.drop(columns=['game_lower', 'name_lower'])

C:\Users\sbm76\AppData\Local\Temp\ipykernel_14964\1218879047.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tags_important['name_lower'] = df_tags_important['name'].str.lower()


In [14]:
# Drop duplicates in merged_df, keeping the first occurrence
merged_df = merged_df.drop_duplicates(subset=['appid', 'game'])

In [15]:
merged_df.shape

(27315, 8)

In [16]:
# Display the first few rows of the merged DataFrame
merged_df.head(27315)

,appid,game,total_votes,upvotes,downvotes,wilson_score,name,genre
0,550,Left 4 Dead 2,1309,1309,0,9.970738e-01,Left 4 Dead 2,[Action]
1,644560,Mirror,925,925,0,9.958641e-01,Mirror,"[Adventure, Indie, RPG]"
2,264710,Subnautica,1333,1332,1,9.957627e-01,Subnautica,"[Adventure, Indie]"
3,294100,RimWorld,1294,1293,1,9.956354e-01,RimWorld,"[Indie, Simulation, Strategy]"
4,413150,Stardew Valley,2240,2236,4,9.954173e-01,Stardew Valley,"[Indie, RPG, Simulation]"
...,...,...,...,...,...,...,...,...
27563,305260,Battle Islands,10,0,10,0.000000e+00,Battle Islands,"[Free to Play, Simulation, Strategy]"
27564,305390,World War 2: Time of Wrath,1,0,1,0.000000e+00,World War 2: Time of Wrath,"[Indie, Strategy]"
27565,305500,Trapped Dead: Lockdown,1,0,1,0.000000e+00,Trapped Dead: Lockdown,"[Action, Indie, RPG]"
27566,2627220,The Texas Chain Saw Massacre - Bloody Killers,1,0,1,0.000000e+00,NaN,NaN


In [17]:
merged_df_needed_cols = merged_df[['appid', 'game', 'genre']]

In [18]:
merged_df_needed_cols

,appid,game,genre
0,550,Left 4 Dead 2,[Action]
1,644560,Mirror,"[Adventure, Indie, RPG]"
2,264710,Subnautica,"[Adventure, Indie]"
3,294100,RimWorld,"[Indie, Simulation, Strategy]"
4,413150,Stardew Valley,"[Indie, RPG, Simulation]"
...,...,...,...
27563,305260,Battle Islands,"[Free to Play, Simulation, Strategy]"
27564,305390,World War 2: Time of Wrath,"[Indie, Strategy]"
27565,305500,Trapped Dead: Lockdown,"[Action, Indie, RPG]"
27566,2627220,The Texas Chain Saw Massacre - Bloody Killers,NaN


In [19]:
# Count the number of rows where 'genre' is missing (NaN)
missing_genre_count = merged_df_needed_cols['genre'].isna().sum()

print(f"Number of entries with missing genre values: {missing_genre_count}")

Number of entries with missing genre values: 17977


Let's make a call to the Steam Storefront API to fill up the missing genres

In [20]:
df_api_call = merged_df_needed_cols.copy(deep=True)

In [38]:
# loading file with completed genres
df_api_call = pd.read_json('df_api_call_completed.json', orient='records', lines=True)

In [39]:
df_api_call

,appid,game,genre
0,550,Left 4 Dead 2,[Action]
1,644560,Mirror,"[Adventure, Indie, RPG]"
2,264710,Subnautica,"[Adventure, Indie]"
3,294100,RimWorld,"[Indie, Simulation, Strategy]"
4,413150,Stardew Valley,"[Indie, RPG, Simulation]"
...,...,...,...
21124,304580,CITYCONOMY: Service for your City,"[Casual, Simulation]"
21125,305260,Battle Islands,"[Free to Play, Simulation, Strategy]"
21126,305390,World War 2: Time of Wrath,"[Indie, Strategy]"
21127,305500,Trapped Dead: Lockdown,"[Action, Indie, RPG]"


Get the genres for all game via the appid

In [24]:
# Load or initialize the cache
cache_file = 'steam_genre_cache_missing_genres.json'
if os.path.exists(cache_file):
    with open(cache_file, 'r') as f:
        cache = json.load(f)
else:
    cache = {}

In [16]:
# Function to fetch genres from Steam Storefront API
def fetch_genres(appid):
    url = f'https://store.steampowered.com/api/appdetails?appids={appid}'
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data[str(appid)]['success']:
                app_data = data[str(appid)]['data']
                genres = [genre['description'] for genre in app_data.get('genres', [])]
                # Fallback: if genres is empty, try categories (often similar to tags)
                if not genres:
                    genres = [tag['description'] for tag in app_data.get('categories', [])]
                return genres
    except Exception as e:
        print(f"Error fetching data for appid {appid}: {e}")
    return []

In [ ]:
# Iterate over each game in the DataFrame
for index, row in tqdm(df_api_call.iterrows(), total=df_api_call.shape[0]):
    appid = str(row['appid'])  # Ensure appid is a string
    
    # Get the genre value
    genre_value = df_api_call.loc[index, 'genre']
    
    # Check if genre is missing or empty
    is_missing = False
    
    # If it's a list, check if it's empty
    if isinstance(genre_value, list):
        if len(genre_value) == 0:
            is_missing = True
    # If it's not a list, check if it's NaN
    elif pd.isna(genre_value):
        is_missing = True
    
    # If genre is missing or empty, get it from cache or API
    if is_missing:
        # Check cache and see if what is returns is empty or not
        if appid in cache and cache[appid]:
            genres = cache[appid]
        else:
            genres = fetch_genres(appid)
            cache[appid] = genres
            # Save cache every 10 entries to prevent data loss
            if len(cache) % 10 == 0:
                with open(cache_file, 'w') as f:
                    json.dump(cache, f)
            time.sleep(1.5)  # Respectful delay to comply with rate limits

        # Update the DataFrame
        df_api_call.at[index, 'genre'] = genres

100%|██████████| 6186/6186 [11:33<00:00,  8.92it/s] 


In [52]:
# Save the final cache
with open(cache_file, 'w') as f:
    json.dump(cache, f)

Attempt to get remaining games via the name of the game

In [25]:
# Replace the missing fetch_genres_or_tags function with this combined function
def fetch_genres_or_tags(appid):
    """
    Fetch genres, falling back to tags if no genres are found.
    """
    url = f'https://store.steampowered.com/api/appdetails?appids={appid}'
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data[str(appid)]['success']:
                app_data = data[str(appid)]['data']
                
                # Try genres first
                genres = [genre['description'] for genre in app_data.get('genres', [])]
                
                # If no genres, try tags instead
                if not genres and 'tags' in app_data:
                    genres = [tag.get('description', tag.get('name', '')) 
                             for tag in app_data.get('tags', [])]
                
                # If still no results, try categories
                if not genres:
                    genres = [cat['description'] for cat in app_data.get('categories', [])]
                    
                return genres
    except Exception as e:
        print(f"Error fetching data for appid {appid}: {e}")
    return []

In [26]:
def search_game_by_name(game_name, max_results=5):
    """
    Search for a game using progressive word removal from the right.
    Example: "Dirt 3" → "Dirt" if first search returns nothing.
    
    Parameters:
        game_name (str): The name of the game to search for
        max_results (int): Maximum number of results to return
        
    Returns:
        list: List of dictionaries with name, app_id, and similarity score
    """
    # Start with full name, then progressively remove words from right
    words = game_name.split()
    search_terms = []
    
    # Generate search terms (original and shortened versions)
    for i in range(len(words), 0, -1):
        term = ' '.join(words[:i])
        search_terms.append(term)
    
    # Try each search term until we get results
    for term in search_terms:
        print(f"Searching for: '{term}'")
        url = f'https://store.steampowered.com/api/storesearch/?term={term}&l=english&cc=US'
        
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                items = data.get('items', [])
                
                if items:
                    # Calculate similarity to original full name
                    matches = []
                    for item in items:
                        steam_name = item.get('name', '')
                        app_id = item.get('id')
                        
                        # Get similarity score
                        similarity = difflib.SequenceMatcher(None, game_name, steam_name).ratio()
                        
                        matches.append({
                            'name': steam_name,
                            'app_id': app_id,
                            'similarity': similarity
                        })
                    
                    # Sort by similarity to original name
                    matches.sort(key=lambda x: x['similarity'], reverse=True)
                    top_matches = matches[:max_results]
                    
                    print(f"Found {len(top_matches)} matches using term '{term}':")
                    for i, match in enumerate(top_matches, 1):
                        print(f"{i}. '{match['name']}' (similarity: {match['similarity']:.2f})")
                    
                    return top_matches
                
                print(f"No results found for '{term}'")
                
        except Exception as e:
            print(f"Error searching for '{term}': {e}")
    
    print(f"Could not find any matches for '{game_name}' after trying all shortened versions")
    return []

In [27]:
def get_genres_by_name(game_name, match_index=0):
    """
    Get genres for a game with fallback to tags if genres are empty.
    If first game match has no genres, we'll use its tags instead.
    
    Parameters:
        game_name (str): The name of the game to search for
        match_index (int): Which match to use (0=best match)
        
    Returns:
        dict or str: Game info including genres or error message
    """
    matches = search_game_by_name(game_name)
    if not matches:
        return f"Could not find game: {game_name}"
    
    if match_index >= len(matches):
        print(f"Warning: Requested match {match_index} but only {len(matches)} found. Using best match.")
        match_index = 0
        
    best_match = matches[match_index]
    app_id = best_match['app_id']
    
    # Get game details
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data[str(app_id)]['success']:
                game_data = data[str(app_id)]['data']
                
                # Get basic info
                name = game_data.get('name', 'Unknown')
                
                # Try genres first
                genres = game_data.get('genres', [])
                genre_list = [genre['description'] for genre in genres]
                
                # If no genres, try tags instead
                if not genre_list and 'tags' in game_data:
                    print(f"No genres found for {name}, using tags instead")
                    genre_list = [tag.get('description', tag.get('name', '')) 
                                 for tag in game_data.get('tags', [])]
                
                return {
                    'name': name,
                    'app_id': app_id,
                    'genres': genre_list
                }
    except Exception as e:
        print(f"Error fetching details: {e}")
    
    return f"Could not fetch details for {game_name}"

In [ ]:
# Iterate over each game in the DataFrame
for index, row in tqdm(df_api_call.iterrows(), total=df_api_call.shape[0]):
    appid = str(row['appid'])  # Ensure appid is a string
    game_name = row['game']  # Get the game name

    # Skip if game name is None or missing
    if game_name is None or pd.isna(game_name):
        print(f"Skipping entry with appid {appid} due to missing game name")
        continue

    # Get the genre value
    genre_value = df_api_call.loc[index, 'genre']
    
    # Check if genre is missing or empty
    is_missing = False
    
    # If it's a list, check if it's empty
    if isinstance(genre_value, list):
        if len(genre_value) == 0:
            is_missing = True
    # If it's not a list, check if it's NaN
    elif pd.isna(genre_value):
        is_missing = True
    
    # If genre is missing or empty, get it from cache or API
    if is_missing:
        genres = []
        # Check cache and ensure the cached genres list is not empty
        if appid in cache and cache[appid]:
            genres = cache[appid]
        else:
            genres = fetch_genres_or_tags(appid)
            # If genres are still empty, attempt to find alternate appid by name
            if not genres:
                print(f"Attempting to find '{game_name}' by name instead of app ID {appid}...")
                matches = search_game_by_name(game_name)
                if matches:
                    best_match = matches[0]
                    new_appid = best_match['app_id']
                    if new_appid and (str(new_appid) != appid):
                        print(f"Found alternate app ID: {new_appid} for game '{game_name}'")
                        genres = fetch_genres_or_tags(new_appid)
            cache[appid] = genres
            # Save cache every 10 entries to prevent data loss
            if len(cache) % 10 == 0:
                with open(cache_file, 'w') as f:
                    json.dump(cache, f)
            time.sleep(1.5)  # Respectful delay to comply with rate limits

        # Update the DataFrame
        df_api_call.at[index, 'genre'] = genres

In [51]:
# Save the final cache
with open(cache_file, 'w') as f:
    json.dump(cache, f)

There are still entries where the game names are missing, and some where the genre list is empty, for now we can placeholder values such as Unknown and Miscellaneous respectively 

In [ ]:
combined_df = df_api_call.copy(deep=True)

In [58]:
# Fix missing game names and genres without adding "misc" to non-empty lists
for index, row in combined_df.iterrows():
    # If game name is missing or NaN
    if pd.isna(row['game']):
        combined_df.at[index, 'game'] = "Unknown"

    # Check if genre exists and is properly formatted
    genre = row['genre']
    
    # Handle different cases of missing or empty genres
    if isinstance(genre, float) and pd.isna(genre):
        # Genre is completely missing (None or scalar NaN)
        combined_df.at[index, 'genre'] = ["Miscellaneous"]
    elif not isinstance(genre, list):
        # Genre is not a list type
        combined_df.at[index, 'genre'] = ["Miscellaneous"]
    elif len(genre) == 0:
        # Genre is an empty list
        combined_df.at[index, 'genre'] = ["Miscellaneous"]
    # If genre is a non-empty list, do nothing

In [59]:
combined_df

,appid,game,genre
0,550,Left 4 Dead 2,[Action]
1,644560,Mirror,"[Adventure, Indie, RPG]"
2,264710,Subnautica,"[Adventure, Indie]"
3,294100,RimWorld,"[Indie, Simulation, Strategy]"
4,413150,Stardew Valley,"[Indie, RPG, Simulation]"
...,...,...,...
27310,1235753,The Sims™ 4 My First Pet Stuff,"[Adventure, Casual, Simulation]"
27311,1240360,The Elder Scrolls V: Skyrim Soundtrack,[Miscellaneous]
27312,1280900,Full Service Soundtrack,[Miscellaneous]
27313,291210,UberStrike,[Miscellaneous]


Save the final dataframe file as json

In [61]:
combined_df.to_json('combined_df.json', orient='records', lines=True)

In [3]:
combined_df = pd.read_json('combined_df.json', orient='records', lines=True)

In [4]:
combined_df

,appid,game,genre
0,550,Left 4 Dead 2,[Action]
1,644560,Mirror,"[Adventure, Indie, RPG]"
2,264710,Subnautica,"[Adventure, Indie]"
3,294100,RimWorld,"[Indie, Simulation, Strategy]"
4,413150,Stardew Valley,"[Indie, RPG, Simulation]"
...,...,...,...
27310,1235753,The Sims™ 4 My First Pet Stuff,"[Adventure, Casual, Simulation]"
27311,1240360,The Elder Scrolls V: Skyrim Soundtrack,[Miscellaneous]
27312,1280900,Full Service Soundtrack,[Miscellaneous]
27313,291210,UberStrike,[Miscellaneous]
